The problem of extracting text from a video can be thought of as a 3 part problem, the first one being, text detection in the Image/Frame,second being orientation of the detected and extracted text and the third part would be the ocr or recognizing text.

### Text Detection [paper](https://arxiv.org/pdf/1911.08947.pdf)
In this pipeline text detection is done using differntial binarization,which is deep CNN with a FPN backbone and it works by predicting the binary map of the Image.

### Text Orientation
text rectification is done using some geometric transformations.

### Text Recognition[paper](https://arxiv.org/pdf/1507.05717.pdf)
Text revognition is done using the CRNN model. This model works by first extracting the Image features using CNN and later passing those features thorugh a Bidirectional LSTM model which then predicts the output word/text. It is trained with CTC loss which helps with non-alignedness of the labels.


The Entire pipeline is alrady put together in the python package [PaddleOcr](https://github.com/PaddlePaddle/PaddleOCR)
This package was used to perform the above task. They have an API which we can directly use for the task.



In [1]:
!python3 -m pip install paddlepaddle==2.0.0 -i https://mirror.baidu.com/pypi/simple

Looking in indexes: https://mirror.baidu.com/pypi/simple
     |████████████████████████████████| 204.9MB 70kB/s 


In [2]:
!pip install moviepy

In [3]:
!pip install paddleocr

     |████████████████████████████████| 133kB 7.7MB/s 
     |████████████████████████████████| 133kB 13.2MB/s 
     |████████████████████████████████| 28.2MB 118kB/s 
     |████████████████████████████████| 2.6MB 52.4MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 2.1MB 48.5MB/s 
     |████████████████████████████████| 81kB 12.9MB/s 
     |████████████████████████████████| 184kB 64.3MB/s 
     |████████████████████████████████| 194kB 60.8MB/s 
     |████████████████████████████████| 51kB 9.0MB/s 
     |████████████████████████████████| 71kB 12.0MB/s 
     |████████████████████████████████| 13.7MB 255kB/s 
     |████████████████████████████████| 7.2MB 47.7MB/s 
     |████████████████████████████████| 645kB 52.2MB/s 
     |████████████████████████████████| 102kB 14.6MB/s 
     |████████████████████████████████| 337kB 60.0MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp36-cp36m-linux_x86_64.whl size=148

In [4]:
import cv2
from paddleocr import PaddleOCR
import os
import time
import concurrent.futures
frames_dir = "/content/im_frames"   #change frames dir here(All the frames extracted will be stored here)
video_path = "/content/drive/MyDrive/If by Rudyard Kipling.mp4" #This is the path to the video

In [5]:
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
ffmpeg_extract_subclip(video_path,28,48 , targetname="test.mp4")


[MoviePy] Running:
>>> /usr/bin/ffmpeg -y -i /content/drive/MyDrive/If by Rudyard Kipling.mp4 -ss 28.00 -t 20.00 -vcodec copy -acodec copy test.mp4
... command successful.


In [6]:
def video_to_frames(input_loc, output_loc):
    """Function to extract frames from input video file
    and save them as separate frames in an output directory.
    Args:
        input_loc: Input video file.
        output_loc: Output directory to save the frames.
    Returns:
        None
    """
    try:
        os.mkdir(output_loc)
    except OSError:
        pass
    # Log the time
    time_start = time.time()
    # Start capturing the feed
    cap = cv2.VideoCapture(input_loc)
    # Find the number of frames
    video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
    print ("Number of frames: ", video_length)
    count = 0
    print ("Converting video..\n")
    # Start converting the video
    while cap.isOpened():
        # Extract the frame
        ret, frame = cap.read()
        # Write the results back to output location.
        cv2.imwrite(output_loc + "/%#05d.jpg" % (count+1), frame)
        count = count + 1
        # If there are no more frames left
        if (count > (video_length-1)):
            # Log the time again
            time_end = time.time()
            # Release the feed
            cap.release()
            # Print stats
            print ("Done extracting frames.\n%d frames extracted" % count)
            print ("It took %d seconds forconversion." % (time_end-time_start))
            break

In [8]:
video_to_frames("test.mp4",frames_dir)

Number of frames:  555
Converting video..

Done extracting frames.
555 frames extracted
It took 9 seconds forconversion.


In [9]:
ocr = PaddleOCR(use_angle_cls=True,lang='en')

Namespace(cls_batch_num=30, cls_image_shape='3, 48, 192', cls_model_dir='/root/.paddleocr/2.0/cls', cls_thresh=0.9, det=True, det_algorithm='DB', det_db_box_thresh=0.5, det_db_thresh=0.3, det_db_unclip_ratio=2.0, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='/root/.paddleocr/2.0/det', drop_score=0.5, enable_mkldnn=False, gpu_mem=8000, image_dir='', ir_optim=True, label_list=['0', '180'], lang='en', max_text_length=25, rec=True, rec_algorithm='CRNN', rec_batch_num=30, rec_char_dict_path='./ppocr/utils/dict/en_dict.txt', rec_char_type='ch', rec_image_shape='3, 32, 320', rec_model_dir='/root/.paddleocr/2.0/rec/en', use_angle_cls=True, use_gpu=True, use_pdserving=False, use_space_char=True, use_tensorrt=False, use_zero_copy_run=False)
download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_det_infer.tar to /root/.paddleocr/2.0/det/ch_ppocr_mobile_v2.0_det_infer.tar


100%|██████████| 3.16M/3.16M [00:06<00:00, 502kiB/s] 


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/multilingual/en_number_mobile_v2.0_rec_infer.tar to /root/.paddleocr/2.0/rec/en/en_number_mobile_v2.0_rec_infer.tar


100%|██████████| 2.69M/2.69M [00:05<00:00, 483kiB/s]


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/2.0/cls/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 1.45M/1.45M [00:09<00:00, 149kiB/s]


In [10]:
def get_ocr(im_path):
  return ocr.ocr(im_path,cls=True)

In [16]:

text_list = []
im_frames = [os.path.join(frames_dir,p) for p in os.listdir(frames_dir)]

for frame in im_frames:
    results = get_ocr(frame)
    for res in results:
        for line in res:
          if line[0] not in text_list:
              text_list.append(line[0])
              print(line[0])


[2021/02/07 12:55:25] root INFO: dt_boxes num : 1, elapse : 0.6265451908111572
[2021/02/07 12:55:25] root INFO: cls num  : 1, elapse : 0.009335041046142578
[2021/02/07 12:55:25] root INFO: rec_res num  : 1, elapse : 0.010662555694580078
[1224.0, 701.0]
Dne
[2021/02/07 12:55:26] root INFO: dt_boxes num : 1, elapse : 0.6219465732574463
[2021/02/07 12:55:26] root INFO: cls num  : 1, elapse : 0.010446548461914062
[2021/02/07 12:55:26] root INFO: rec_res num  : 1, elapse : 0.006219625473022461
[2021/02/07 12:55:26] root INFO: dt_boxes num : 2, elapse : 0.6245825290679932
[2021/02/07 12:55:26] root INFO: cls num  : 2, elapse : 0.014412403106689453
[2021/02/07 12:55:26] root INFO: rec_res num  : 2, elapse : 0.06200432777404785
[291.0, 342.0]
are losing theirs and blaming it onyou
[1214.0, 695.0]
illneas
[2021/02/07 12:55:27] root INFO: dt_boxes num : 2, elapse : 0.6215524673461914
[2021/02/07 12:55:27] root INFO: cls num  : 2, elapse : 0.016356706619262695
[2021/02/07 12:55:27] root INFO: rec

In [27]:
f = open("text_vid.txt","w")
for l in text_list:
  if type(l) is str:
    l = l+ "\n"
    f.write(l)
f.close()
